In [4]:
# -*- coding: utf-8 -*-
# @author: caoyang
# @email: caoyang@163.sufe.edu.cn

import os
import re
import time
import requests
import random

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup

def get_cookie(url):
	options = webdriver.FirefoxOptions()								
	options.add_argument("--headless")									
	driver = webdriver.Firefox(options=options)							
	driver.get(url)
	cookies = driver.get_cookies()
	driver.quit()
	def _cookie_to_string(cookies):
		string = ''
		for cookie in cookies:
			string += '{}={}; '.format(cookie['name'], cookie['value'])
		return string.strip()
	return _cookie_to_string(cookies)
	

def download_chinese_statistical_yearbook(ybcode='N2020100004', year='2020', loc = '12',save_root='csyb', is_initial=True, ignore_caj=True):
	with open('system_csyb.log', 'w') as f:
		pass
	headers = {'User-Agent': 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0'}
	query_url = 'https://data.cnki.net/Yearbook/PartialGetCatalogResult'
	excel_url = 'https://data.cnki.net/{}'.format
	caj_url = 'https://data.cnki.net/download/GetCajUrl'
	regex = r'<[^>]+>'
	cookies = get_cookie('https://data.cnki.net/trade/Yearbook/Single/{}?z=Z016'.format(ybcode))
	compiler = re.compile(regex, re.S)
	regular_interval = 15
	reset_interval = 300
	

	if not os.path.exists(save_root):
		os.mkdir(save_root)
	# year = ybcode[1:5]
	target_path = os.path.join(save_root, year)
	if not os.path.exists(target_path):
		os.mkdir(target_path)

	with open(os.path.join(target_path, 'log.txt'), 'w') as f:
		pass

	formdata = {
		'ybcode': ybcode,
		'entrycode': '',
		'page': '1',
		'pagerow': '20'
	}
	response = requests.post(query_url, data=formdata, headers=headers)
	html = response.text
	soup = BeautifulSoup(html, 'lxml')
	span = soup.find('span', class_='s_p_listl')
	for link in span.find_all('a'):
		onclick = link.attrs.get('onclick')
		if onclick is not None:
			lindex = onclick.find('\'')
			rindex = onclick.find('\'', lindex + 1)
			n_pages = int(onclick[lindex + 1:rindex])
			break
	with open('system_csyb.log', 'a') as f:
		f.write('正在处理{}年...\t{}\n'.format(year, time.strftime('%Y-%m-%d %H:%M:%S')))
	print('正在处理{}年...'.format(year))
	with open('system_csyb.log', 'a') as f:
		f.write('共计{}页\t{}\n'.format(n_pages, time.strftime('%Y-%m-%d %H:%M:%S')))
	print('共计{}页'.format(n_pages))
	for page in range(1, n_pages + 1):
		with open('system_csyb.log', 'a') as f:
			f.write('  - 第{}页..\t{}\n'.format(page, time.strftime('%Y-%m-%d %H:%M:%S')))
		print('  - 第{}页..'.format(page))
		if not page == '1': 
			formdata = {
				'ybcode': ybcode,
				'entrycode': '',
				'page': str(page),
				'pagerow': '20'
			}
			while True:
				try:
					response = requests.post(query_url, data=formdata, headers=headers)
					break
				except:
					with open('system_csyb.log', 'a') as f:
						f.write('    页面访问失败...\t{}\n'.format(time.strftime('%Y-%m-%d %H:%M:%S')))
					print('    页面访问失败...')
					time.sleep(reset_interval)				
			html = response.text
			soup = BeautifulSoup(html, 'lxml')
		table = soup.find('table')
		for tr in table.find_all('tr'):
			tds = tr.find_all('td')
			assert len(tds) == 3
			title = compiler.sub('', str(tds[0])).replace('\n', '').replace('\t', '').replace(' ', '').replace('\r', '')
			page_range = compiler.sub('', str(tds[1])).replace('\n', '').replace('\t', '').replace(' ', '')
			if title[0:2]!=loc:
    				continue
			for _link in tds[2].find_all('a'):
				href = _link.attrs['href']
				if href.startswith('/download/excel'):					 # excel
					filecode = href[href.find('=')+1:]
					while True:
						_headers = headers.copy()
						_headers['Cookie'] = cookies
						try:
							with open('system_csyb.log', 'a') as f:
								f.write('    + 下载{}...\t{}\n'.format(title, time.strftime('%Y-%m-%d %H:%M:%S')))
							print('    + 下载{}...'.format(title))
							response = requests.get(excel_url(href), headers=_headers)
							st = random.randint(3,5) + random.random()
							time.sleep(st)
							print('      ' + str(response.status_code))
							try:
								html = response.text
								soup = BeautifulSoup(html, 'lxml')
								if str(soup.find('title').string)=='中国经济社会大数据研究平台':
									with open('system_csyb.log', 'a') as f:
										f.write('      重置cookie...\t{}\n'.format(time.strftime('%Y-%m-%d %H:%M:%S')))
									print('      重置cookie...')
									cookies = get_cookie('https://data.cnki.net/trade/Yearbook/Single/{}?z=Z016'.format(ybcode))
								else:
									break
							except:
								break
						except:
							with open('system_csyb.log', 'a') as f:
								f.write('      失败...\t{}\n'.format(time.strftime('%Y-%m-%d %H:%M:%S')))
							print('      失败...')
							time.sleep(reset_interval)
							cookies = get_cookie('https://data.cnki.net/trade/Yearbook/Single/{}?z=Z016'.format(ybcode))
					time.sleep(regular_interval)
					
					with open(os.path.join(target_path, '{}.xls'.format(filecode)), 'wb') as f:
						f.write(response.content)
					with open(os.path.join(target_path, 'log.txt'), 'a') as f:
						f.write('{}\t{}\t{}.xls\n'.format(title, page_range, filecode))
				else:													 # caj
					if ignore_caj:
						continue
					filecode = _link.attrs['fn']
					pagerange = _link.attrs['pg']
					disk = _link.attrs['disk']
					_formdata = {
						'filecode': filecode,
						'pagerange': pagerange,
						'disk': disk,
					}
					while True:
						_headers = headers.copy()
						_headers['Cookie'] = cookies	
						try:		
							with open('system_csyb.log', 'a') as f:
								f.write('    + 下载{}的资源链接...\t{}\n'.format(title, time.strftime('%Y-%m-%d %H:%M:%S')))
							print('    + 下载{}的资源链接...'.format(title))		
							response = requests.post(caj_url, headers=_headers, data=_formdata)
							break
						except:
							with open('system_csyb.log', 'a') as f:
								f.write('      失败...\t{}\n'.format(time.strftime('%Y-%m-%d %H:%M:%S')))
							print('      失败...')
							time.sleep(reset_interval)		
							cookies = get_cookie('https://data.cnki.net/trade/Yearbook/Single/{}?z=Z016'.format(ybcode))					
					resource_url = response.json()['url']
					while True:
						try:
							with open('system_csyb.log', 'a') as f:
								f.write('    + 下载{}...\t{}\n'.format(title, time.strftime('%Y-%m-%d %H:%M:%S')))
							print('    + 下载{}...'.format(title))
							response = requests.get(resource_url, headers=headers)
							st = random.randint(3,5) + random.random()
							time.sleep(st)
							if str(response.status_code) == '200':
								break 
							else:
								with open('system_csyb.log', 'a') as f:
									f.write('      重置cookie...\t{}\n'.format(time.strftime('%Y-%m-%d %H:%M:%S')))
								print('      重置cookie...')
								time.sleep(reset_interval)
								cookies = get_cookie('https://data.cnki.net/trade/Yearbook/Single/{}?z=Z016'.format(ybcode))
						except:
							with open('system_csyb.log', 'a') as f:
								f.write('      失败...\t{}\n'.format(time.strftime('%Y-%m-%d %H:%M:%S')))
							print('      失败...')
							time.sleep(regular_interval)
							cookies = get_cookie('https://data.cnki.net/trade/Yearbook/Single/{}?z=Z016'.format(ybcode))
					time.sleep(regular_interval)
					with open(os.path.join(target_path, '{}.caj'.format(filecode)), 'wb') as f:
						f.write(response.content)
					with open(os.path.join(target_path, 'log.txt'), 'a') as f:
						f.write('{}\t{}\t{}.caj\n'.format(title, page_range, filecode))

			
	# Find urls of year
	if is_initial:
		url = 'https://data.cnki.net/trade/Yearbook/Single/{}?z=Z016'.format(ybcode)
		response = requests.get(url, headers=headers)
		st = random.randint(3,5) + random.random()
		time.sleep(st)
		html = response.text
		soup = BeautifulSoup(html, 'lxml')
		div = soup.find('div', class_='s_year clearfix')
		links = []
		ybcodes = []
		for link in div.find_all('a'):
			class_ = link.attrs.get('class')
			if class_ is None:											 # not current
				href = link.attrs.get('href')
				ybcode = href.split('/')[-1].split('?')[0]
				links.append(href)
				ybcodes.append(ybcode)
		with open('ybcode_csyb.txt', 'w') as f:
			for ybcode in ybcodes:
				f.write(f'{ybcode}\n')
		# for ybcode in ybcodes:
		#	download_chinese_statistical_yearbook(ybcode=ybcode, is_initial=False)
	





if __name__ == '__main__':
	with open('ybcode_csyb.txt', 'r') as f:
	
		lines = f.read().splitlines()	
		print(lines)
	for line in lines:
		ybcode, year, loc = line.split()
		# if int(year) > 1999:
			# continue
		download_chinese_statistical_yearbook(ybcode=ybcode, year=year, loc = loc ,is_initial=False)
		st = random.randint(60,100) + random.random()
		time.sleep(st)
		



['N2017030066 2016 12', 'N2015110123 2015 12', 'N2014120083 2014 12', 'N2013110049 2013 13', 'N2012100019 2012 13', 'N2011090108 2011 13', 'N2010100096 2010 13', 'N2010030101 2009 12', 'N2009030129 2008 12', 'N2008090019 2007 13', 'N2007091091 2006 13', 'N2006060155 2005 13', 'N2006010336 2004 13', 'N2006010337 2003 12', 'N2005121054 2002 12', 'N2006010338 2001 12', 'N2005120828 2000 12', 'N2006010671 1999 12', 'N2005120325 1998 12', 'N2005120241 1997 11', 'N2005120323 1996 11', 'N2005120324 1995 11', 'N2005120242 1994 11', 'N2005120238 1993 9-', 'N2005120322 1992 9-', 'N2005120321 1991 9-', 'N2005121062 1990 9-', 'N2005120243 1989 5-', 'N2005121060 1988 5-', 'N2005120239 1987 4-', 'N2005120240 1986 4-', 'N2005120319 1985 3-', 'N2005120320 1984 3-', 'N2006060137 1983 3-', 'N2006060137 1981 3-']
正在处理2016年...
共计41页
  - 第1页..
  - 第2页..
  - 第3页..
  - 第4页..
  - 第5页..
  - 第6页..
  - 第7页..
  - 第8页..
  - 第9页..
  - 第10页..
  - 第11页..
  - 第12页..
  - 第13页..
  - 第14页..
    + 下载12-1农业生产条件与农作物播种面积...


KeyboardInterrupt: 